In [1]:
import torch
# import torchvision
import torch.nn as nn
import matplotlib
import matplotlib.pyplot as plt
# import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import os
import time
import numpy as np
import argparse
from tqdm import tqdm
# from torchvision import datasets
from torch.utils.data import DataLoader,TensorDataset
# from torchvision.utils import save_image
matplotlib.style.use('ggplot')

In [2]:
# constructing argument parsers 
# ap = argparse.ArgumentParser()
# ap.add_argument('-e', '--epochs', type=int, default=5,
#     help='number of epochs to train our network for')
# ap.add_argument('-l', '--reg_param', type=float, default=0.001, 
#     help='regularization parameter `lambda`')
# ap.add_argument('-sc', '--add_sparse', type=str, default='yes', 
#     help='whether to add sparsity contraint or not')
# args = vars(ap.parse_args())

# EPOCHS = args['epochs']
# BETA = args['reg_param']
# ADD_SPARSITY = args['add_sparse']
# RHO = 0.05
# LEARNING_RATE = 1e-3
# BATCH_SIZE = 32
# print(f"Add sparsity regularization: {ADD_SPARSITY}")

In [3]:
# get the computation device
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
device = 'cuda:0'


In [4]:
import keras
import keras.backend as K
from keras.layers import LSTM,Dropout,Dense,TimeDistributed,Conv1D,MaxPooling1D,Flatten
from keras.models import Sequential
import tensorflow as tf
import gc
from numba import jit
from IPython.display import display, clear_output
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
sns.set_style("whitegrid")

In [5]:
from tensorflow.keras.metrics import AUC
def keras_auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [6]:
n_signals = 4
n_outputs = 1 #Binary Classification

In [7]:
'''
Need to get data into a DataLoader 
'''
i = 1
# train_data = np.loadtxt("./data/processed/standardized_train_FD00"+str(i)+".txt",delimiter=",")
# test_data = np.loadtxt("./data/processed/standardized_test_FD00"+str(i)+".txt",delimiter=",")
train_data = np.loadtxt("standardized_train_FD00"+str(i)+".txt",delimiter=",")
test_data = np.loadtxt("standardized_train_FD00"+str(i)+".txt",delimiter=",")
X_train = torch.Tensor(train_data[:,2:])
y_train = torch.Tensor(train_data[:,1]).reshape((-1,1))
X_test = torch.Tensor(test_data[:,2:])
y_test = torch.Tensor(test_data[:,1])

train_dataset = TensorDataset(X_train,y_train)
train_dataloader = DataLoader(train_dataset)
test_dataset = TensorDataset(X_test,y_test)

In [8]:
from tensorflow.keras.layers import LeakyReLU
#Build the model
# verbose, epochs, batch_size = True, 15, 16
verbose, epochs, batch_size = True, 15, 16
# n_steps, n_length = 4, 6
n_steps, n_length = 1, 6
# n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, n_signals))
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_signals)))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))

# model = Sequential()
# model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(None,n_length,n_signals))))
# model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
# model.add(TimeDistributed(Flatten()))
# model.add(TimeDistributed(Dropout(0.5)))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32)

# # Evaluate the model
# loss, accuracy = model.evaluate(X_test, y_test)
# print('Test accuracy:', accuracy)
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
from tensorflow.keras.metrics import AUC
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC()])

In [10]:
X_train = tf.cast(X_train, dtype = tf.float32)
X_test = tf.cast(X_test, dtype = tf.float32)
y_train = tf.cast(y_train, dtype = tf.float32)
y_test = tf.cast(y_test, dtype = tf.float32)

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=verbose)

Epoch 1/10
1290/1290 [==============================] - 4s 2ms/step - loss: -677.4311 - accuracy: 0.3497 - auc: 0.5829
Epoch 2/10
1290/1290 [==============================] - 2s 2ms/step - loss: -4663.9634 - accuracy: 0.3581 - auc: 0.6001
Epoch 3/10
1290/1290 [==============================] - 2s 2ms/step - loss: -11816.7363 - accuracy: 0.3524 - auc: 0.5564
Epoch 4/10
1290/1290 [==============================] - 2s 2ms/step - loss: -21806.5273 - accuracy: 0.3414 - auc: 0.5725
Epoch 5/10
1290/1290 [==============================] - 2s 2ms/step - loss: -34418.9336 - accuracy: 0.3364 - auc: 0.5625 ETA: 1s - loss: -2730
Epoch 6/10
1290/1290 [==============================] - 2s 2ms/step - loss: -49233.0820 - accuracy: 0.3321 - auc: 0.5950
Epoch 7/10
1290/1290 [==============================] - 2s 2ms/step - loss: -66472.1172 - accuracy: 0.3336 - auc: 0.5823
Epoch 8/10
1290/1290 [==============================] - 2s 2ms/step - loss: -85503.9531 - accuracy: 0.3308 - auc: 0.5790
Epoch 9/10
12

In [17]:
# Evaluate the model
import torch
X_train = torch.Tensor(train_data[:,2:])
X_test = torch.Tensor(test_data[:,2:])
X_test = X_test.reshape((X_train.shape[0], n_steps, n_length, n_signals))
y_test = torch.Tensor(test_data[:,1])
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)
print('Test accuracy:', accuracy)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

TypeError: Expected DataType for argument 'Tout' not torch.float32.

In [ ]:
import torch.nn.functional as F
def encode(x):
        x = F.relu(x)
        x = F.relu(x)
        return x

test_dataloader = DataLoader(test_dataset)
features_train = np.zeros((train_data.shape[0],26))
features_test = np.zeros((test_data.shape[0],26))
features_train[:,0:2] = train_data[:,0:2]
features_test[:,0:2] = test_data[:,0:2]

with torch.no_grad():
    for i, data in tqdm(enumerate(train_dataloader), total=int(len(train_dataset)/train_dataloader.batch_size)):
        pt, _ = data
        pt = pt.to(device)
        pt = pt.view(pt.size(0), -1)
        outputs = encode(pt).cpu()
        # features_train[i,2:] = outputs
        features_train[i,2:] = outputs
    for i, data in tqdm(enumerate(test_dataloader), total=int(len(test_dataset)/test_dataloader.batch_size)):
        pt, _ = data
        pt = pt.to(device)
        pt = pt.view(pt.size(0), -1)
        outputs = np.array(encode(pt).cpu())
        # print(outputs)
        features_train[i,2:] = outputs
print("Features processed, now saving to file")


np.savetxt("cnn_traindata.txt",features_train,delimiter=",")
np.savetxt("cnn_testdata.txt",features_test,delimiter=",")

In [ ]:
val = model.predict(X_train)
val